# Notebook 02 — Preprocessing zu `model_ready.csv` (Anforderungen 4–5)

Dieses Notebook deckt folgende Anforderungen ab:
- **Anforderung 4:** Datenvorverarbeitung (Scope, QC, Feature Engineering)
- **Anforderung 5 (optional):** Data Augmentation (nicht angewendet)

Ziel:
- Anwendung des Projekt-Scopes (nur Fahrten **≥ 5 km**)
- Feature Engineering für die Modellierung
- Erstellung der zentralen Modellgrundlage `model_ready.csv`

Leakage-Regel:
- `elapsed_time` bleibt als QC-Spalte im Datensatz vorhanden, wird später aber **nicht als Feature** verwendet.

Outputs:
- `data/processed/model_ready.csv`
- `data_derived/02_scope_filter_report.csv`
- `data_derived/02_feature_engineering_report.csv`
- `data_derived/02_qc_summary_model_ready.csv`
- `data_derived/02_feature_list_model_ready.csv`

In [1]:
from __future__ import annotations

from pathlib import Path
import numpy as np
import pandas as pd

SEED = 42
np.random.seed(SEED)

def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start] + list(start.parents):
        if (p / "data" / "processed").exists():
            return p
    return start

REPO_ROOT = find_repo_root(Path.cwd())

PATH_PROCESSED = REPO_ROOT / "data" / "processed" / "processed.csv"
PATH_MODEL_READY = REPO_ROOT / "data" / "processed" / "model_ready.csv"
PATH_DERIVED = REPO_ROOT / "data_derived"

PATH_MODEL_READY.parent.mkdir(parents=True, exist_ok=True)
PATH_DERIVED.mkdir(parents=True, exist_ok=True)

TARGET = "moving_time"
LEAKAGE_COLS = ["elapsed_time"]

def safe_read_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(
            "[ERROR] Datei nicht gefunden.\n"
            f"Erwarteter Pfad: {path}\n"
            "Bitte Notebook 01 ausführen (processed.csv erzeugen)."
        )
    return pd.read_csv(path)

print("REPO_ROOT:", REPO_ROOT)
print("PATH_PROCESSED:", PATH_PROCESSED)
print("PATH_MODEL_READY:", PATH_MODEL_READY)
print("PATH_DERIVED:", PATH_DERIVED)

REPO_ROOT: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer
PATH_PROCESSED: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data/processed/processed.csv
PATH_MODEL_READY: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data/processed/model_ready.csv
PATH_DERIVED: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived


In [2]:
df = safe_read_csv(PATH_PROCESSED)
print("processed geladen:", df.shape)

required = ["distance", "total_elevation_gain", "highest_elevation", "lowest_elevation", "moving_time"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"[ERROR] Erwartete Spalten fehlen in processed.csv: {missing}")

# distance in km (robust, falls schon km)
df["distance_km"] = df["distance"] / 1000.0 if df["distance"].max() > 500 else df["distance"]

report_rows = []
report_rows.append({"step": "load_processed", "rows": len(df), "removed": np.nan})

before = len(df)
df = df[df["distance_km"] >= 5.0].copy()
report_rows.append({"step": "scope_filter_distance_km_ge_5", "rows": len(df), "removed": before - len(df)})

scope_report = pd.DataFrame(report_rows)
display(scope_report)

scope_path = PATH_DERIVED / "02_scope_filter_report.csv"
scope_report.to_csv(scope_path, index=False)
print("Saved:", scope_path)

print("min distance_km after scope:", df["distance_km"].min())
print("shape after scope:", df.shape)

processed geladen: (9948, 6)


,step,rows,removed
0,load_processed,9948,NaN
1,scope_filter_distance_km_ge_5,9237,711.0


Saved: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived/02_scope_filter_report.csv
min distance_km after scope: 5.001399999999999
shape after scope: (9237, 7)


## Feature Engineering und Qualitätschecks (QC)

Wir erweitern den Datensatz um wenige, nachvollziehbare Features:

- `elev_range` = `highest_elevation - lowest_elevation`
- `elev_gain_per_km` = `total_elevation_gain / distance_km`
- `stop_time` = `elapsed_time - moving_time` (nur wenn `elapsed_time` vorhanden)
- `stop_ratio` = `stop_time / elapsed_time` (nur wenn `elapsed_time` vorhanden)
- `pace_min_per_km` = Fahrtdauer pro km in Minuten
- `avg_speed_implied_kmh` = Durchschnittsgeschwindigkeit aus `distance_km` und `moving_time`

Anschließend führen wir einen kompakten QC-Check aus (Duplicates, Missing, Non-finite).

In [3]:
fe_report = []

# elev_range
df["elev_range"] = df["highest_elevation"] - df["lowest_elevation"]
fe_report.append({"feature": "elev_range", "definition": "highest_elevation - lowest_elevation"})

# elev_gain_per_km (Division-by-zero durch scope >=5km ausgeschlossen)
df["elev_gain_per_km"] = df["total_elevation_gain"] / df["distance_km"]
fe_report.append({"feature": "elev_gain_per_km", "definition": "total_elevation_gain / distance_km"})

# stop_time / stop_ratio nur, falls elapsed_time vorhanden
if "elapsed_time" in df.columns:
    df["stop_time"] = df["elapsed_time"] - df["moving_time"]
    df["stop_ratio"] = df["stop_time"] / df["elapsed_time"]
    fe_report.append({"feature": "stop_time", "definition": "elapsed_time - moving_time"})
    fe_report.append({"feature": "stop_ratio", "definition": "stop_time / elapsed_time"})

# pace_min_per_km
df["moving_time_min"] = df["moving_time"] / 60.0
df["pace_min_per_km"] = df["moving_time_min"] / df["distance_km"]
fe_report.append({"feature": "pace_min_per_km", "definition": "(moving_time/60) / distance_km"})

# avg_speed_implied_kmh
df["avg_speed_implied_kmh"] = df["distance_km"] / (df["moving_time"] / 3600.0)
fe_report.append({"feature": "avg_speed_implied_kmh", "definition": "distance_km / (moving_time/3600)"})

print("Shape after FE:", df.shape)

fe_df = pd.DataFrame(fe_report)
display(fe_df)

fe_path = PATH_DERIVED / "02_feature_engineering_report.csv"
fe_df.to_csv(fe_path, index=False)
print("Saved:", fe_path)

Shape after FE: (9237, 14)


,feature,definition
0,elev_range,highest_elevation - lowest_elevation
1,elev_gain_per_km,total_elevation_gain / distance_km
2,stop_time,elapsed_time - moving_time
3,stop_ratio,stop_time / elapsed_time
4,pace_min_per_km,(moving_time/60) / distance_km
5,avg_speed_implied_kmh,distance_km / (moving_time/3600)


Saved: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived/02_feature_engineering_report.csv


In [4]:
# QC Summary
qc = pd.DataFrame([{
    "n_rows": int(df.shape[0]),
    "n_cols": int(df.shape[1]),
    "n_duplicates_all_cols": int(df.duplicated().sum()),
    "missing_cells_total": int(df.isna().sum().sum()),
    "missing_rows_any": int(df.isna().any(axis=1).sum()),
    "has_elapsed_time_col": int("elapsed_time" in df.columns),
    "target_col_present": int(TARGET in df.columns),
}])

display(qc)

qc_path = PATH_DERIVED / "02_qc_summary_model_ready.csv"
qc.to_csv(qc_path, index=False)
print("Saved:", qc_path)

# Feature-Liste für Modellierung (inkl. Leakage-Spalten separat dokumentieren)
all_cols = df.columns.tolist()
feature_cols = [c for c in all_cols if c != TARGET]

feature_list = pd.DataFrame({"feature": feature_cols})
feat_path = PATH_DERIVED / "02_feature_list_model_ready.csv"
feature_list.to_csv(feat_path, index=False)
print("Saved:", feat_path)

# model_ready schreiben (wir behalten elapsed_time im Datensatz; spätere Notebooks nutzen ihn nicht als Feature)
df_out = df.drop(columns=["moving_time_min"], errors="ignore").copy()
df_out.to_csv(PATH_MODEL_READY, index=False)
print("Saved:", PATH_MODEL_READY)

print("model_ready shape:", df_out.shape)

,n_rows,n_cols,n_duplicates_all_cols,missing_cells_total,missing_rows_any,has_elapsed_time_col,target_col_present
0,9237,14,82,0,0,1,1


Saved: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived/02_qc_summary_model_ready.csv
Saved: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived/02_feature_list_model_ready.csv
Saved: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data/processed/model_ready.csv
model_ready shape: (9237, 13)
